In [5]:
# Install dependencies
!pip install pandas numpy scikit-learn
!pip install transformers torch xgboost
!pip install sortedcollections


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
from pathlib import Path
import numpy as np
import sys
import os
import warnings
from pathlib import Path
from sklearn.pipeline import make_pipeline

# Get the current working directory
current_dir = os.getcwd()

# Assuming the current directory is 'models', get the path of the 'TAQ-Query-Scripts' directory
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

# Now, you can import your modules from spark_pipelines
from spark_pipelines.generators import *
from spark_pipelines.clean_data import *
from spark_pipelines.preprocess_data import *
from spark_pipelines.feature_generation import *

In [2]:
from transformers import AutoModel
from huggingface_hub import HfFolder
from dotenv import load_dotenv
import os

token = os.getenv("hf_token")

# Use the token to authenticate
HfFolder.save_token(token)

# Now you can load your private model
#model = AutoModel.from_pretrained("taqdatabase/OLS")

In [5]:
# Import necessary libraries
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import joblib

# Load the dataset
df_test = pd.read_csv('sample_features.csv')
df_test['Trade_Price'] = df_test['Trade_Price'].fillna(df_test['Trade_Price'].median())

# Selecting features for the model (adjust as necessary)
features = ['Trade_Volume', 'Bid_Price', 'Offer_Price']  # Example features
target = 'Trade_Price'

# Drop rows with missing target or features
df_test = df_test.dropna(subset=[target] + features)

# Check if the DataFrame is empty
if df_test.empty:
    print("No data available after removing missing values.")
else:
    # Define X (features) and y (target)
    X = df_test[features]
    y = df_test[target]

    # Adding a constant to the model (important for OLS regression)
    X = sm.add_constant(X)

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create the OLS model
    model = sm.OLS(y_train, X_train).fit()

    # Print model summary
    print(model.summary())

    # Save the model to a file
    model_filename = 'ols_model.pkl'
    joblib.dump(model, model_filename)

    print(f"Model saved as {model_filename}")

                            OLS Regression Results                            
Dep. Variable:            Trade_Price   R-squared:                       0.600
Model:                            OLS   Adj. R-squared:                  0.599
Method:                 Least Squares   F-statistic:                     596.1
Date:                Fri, 05 Jan 2024   Prob (F-statistic):          7.01e-159
Time:                        16:16:20   Log-Likelihood:                 23018.
No. Observations:                 798   AIC:                        -4.603e+04
Df Residuals:                     795   BIC:                        -4.602e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          294.6500   1.46e-12   2.02e+14   